In [ ]:
import numpy as np
from logistic_regression.GrdDscntQuant import grdescentquant
from logistic_regression.normal_logistic import normallogistic
from logistic_regression.GrdDscnt import grdescentnormal
from logistic_regression.quant_logistic import quant_logistic
from logistic_regression.GrdDscntUncoded import grdescentuncoded
from logistic_regression.uncoded_logistic import  uncoded_logistic
from quantization.quantize import quantize
from coded_computation.master import master
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from pad_and_clean import pad
from pad_and_clean import clean_and_scale
import time
from gen_data import gen_data, gen_nonlinear_data
from sklearn.model_selection import train_test_split
import numpy as np
from logistic_regression.GrdDscntQuant import grdescentquant
from logistic_regression.normal_logistic import normallogistic
from logistic_regression.GrdDscnt import grdescentnormal
from logistic_regression.quant_logistic import quant_logistic
from logistic_regression.GrdDscntUncoded import grdescentuncoded
from logistic_regression.uncoded_logistic import  uncoded_logistic
from quantization.quantize import quantize
from coded_computation.master import master
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from pad_and_clean import pad
from pad_and_clean import clean_and_scale
import time
from gen_data import gen_data, gen_nonlinear_data
from sklearn.model_selection import train_test_split

def get_loss(w,X,y):
    #calculates 1-0 prediction error
    log_odds = X@w
    probs = 1 / (1 + np.exp(-log_odds))
    preds = np.where(probs > 0.5, 1,-1)
    test_loss = np.mean(preds != y)

    return test_loss
def plot_3d_bar(data, z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Define the x, y coordinates and the z heights
    _x = np.arange(data.shape[1])
    _y = np.arange(data.shape[0])
    _x, _y = np.meshgrid(_x, _y)
    x, y = _x.ravel(), _y.ravel()

    # The z values represent the bar heights
    z = np.zeros_like(x)
    dz = data.ravel()

    # Plot 3D bars
    ax.bar3d(x, y, z, 1, 1, dz, shade=True)

    # Labels
    ax.set_xlabel('w_lvl')
    ax.set_ylabel('grd_lvl')
    ax.set_zlabel(z)

    plt.show()

def run(X,y, filename):
    func = quant_logistic
    G = np.array([
        [1, 1, 1, 1, 1, 1, 1],
        [-1, -1, -1, 1, 1, 1, 1],
        [-1, 1, 1, -1, -1, 1, 1],
        [1, -1, -1, -1, -1, 1, 1],
        [1, -1, 1, -1, 1, -1, 1],
        [-1, 1, -1, -1, 1, -1, 1],
        [-1, -1, 1, 1, -1, -1, 1],
        [1, 1, -1, 1, -1, -1, 1]
    ]).T

    stepsize = 0.1
    maxiter = 10000

    #X,y = gen_nonlinear_data(500, 40, 1)

    X, Xt, y, yt = train_test_split(X, y, test_size=0.2)
    X, y = pad(X, y, 7)
    Xt, yt = pad(Xt, yt, 7)

    Master_uncoded = master(X, None, 21)
    Master = master(X, G, 3)
    times_grid = np.zeros((5, 4))
    loss_grid = np.zeros((5, 4))  # Initialize for each dataset
    iters_grid = np.zeros((5, 4))
    test_loss = np.zeros((5, 4))
    test_loss_uncoded = np.zeros((5,4))
    uncoded_times = np.zeros((5, 4))
    # loss from normal logistic regression
    w0 = np.random.uniform(-1, 1, (X.shape[1], 1))
    start_time = time.time()
    w, num_iters = grdescentnormal(normallogistic, w0, stepsize, maxiter, Master_uncoded, y, X, tolerance=1e-02)
    normal_loss = get_loss(w, Xt, yt)
    end_time = time.time()
    print(f"loss from unquantized logistic regresison: {normal_loss} on {num_iters} iterations in {end_time - start_time} seconds")
    repetitions = 40
    for i in range(repetitions):
        for w_lvl in range(4, 9):
            for grd_lvl in range(2, 6):
                # logic for calculations
                w0 = np.random.uniform(-1, 1, (X.shape[1], 1))

                start_time = time.time()
                #w, num_iters = grdescentquant(func, w0, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, tolerance=1e-02)
                w, num_iters = grdescentquant(func, w0, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, 1e-02)
                # grdescentquant(func, w, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, tolerance)

                end_time = time.time()

                times_grid[w_lvl - 4, grd_lvl - 2] += end_time - start_time
                loss_grid[w_lvl - 4, grd_lvl - 2] += get_loss(w, X, y)
                iters_grid[w_lvl - 4, grd_lvl - 2] += num_iters
                test_loss[w_lvl - 4, grd_lvl - 2] += get_loss(w, Xt, yt)

                start_time = time.time()
                w, num_iters = grdescentuncoded(uncoded_logistic, w0, stepsize, maxiter, Master_uncoded, w_lvl, grd_lvl, X, y, tolerance=1e-02)
                end_time = time.time()
                uncoded_times[w_lvl - 4, grd_lvl - 2] += end_time - start_time
                test_loss_uncoded[w_lvl - 4, grd_lvl - 2] += get_loss(w,Xt,yt)

    test_loss = test_loss/repetitions
    loss_grid = loss_grid/repetitions
    iters_grid = iters_grid/repetitions
    times_grid = times_grid/repetitions
    test_loss_uncoded = test_loss_uncoded/repetitions
    uncoded_times = uncoded_times/repetitions



    print(f" training loss: \n{loss_grid}")
    print(f" avg iterations: \n{iters_grid}")
    print(f" test loss: \n{test_loss}")
    print(f"run time: \n{times_grid}")
    print(f" test loss uncoded: \n{test_loss_uncoded}")
    print(f" times uncoded: \n{uncoded_times}")
    return test_loss, loss_grid, iters_grid, times_grid, test_loss_uncoded, uncoded_times
def get_loss(w,X,y):
    #calculates 1-0 prediction error
    log_odds = X@w
    probs = 1 / (1 + np.exp(-log_odds))
    preds = np.where(probs > 0.5, 1,-1)
    test_loss = np.mean(preds != y)

    return test_loss
def plot_3d_bar(data, z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Define the x, y coordinates and the z heights
    _x = np.arange(data.shape[1])
    _y = np.arange(data.shape[0])
    _x, _y = np.meshgrid(_x, _y)
    x, y = _x.ravel(), _y.ravel()

    # The z values represent the bar heights
    z = np.zeros_like(x)
    dz = data.ravel()

    # Plot 3D bars
    ax.bar3d(x, y, z, 1, 1, dz, shade=True)

    # Labels
    ax.set_xlabel('w_lvl')
    ax.set_ylabel('grd_lvl')
    ax.set_zlabel(z)

    plt.show()

def run(X,y, filename):
    func = quant_logistic
    G = np.array([
        [1, 1, 1, 1, 1, 1, 1],
        [-1, -1, -1, 1, 1, 1, 1],
        [-1, 1, 1, -1, -1, 1, 1],
        [1, -1, -1, -1, -1, 1, 1],
        [1, -1, 1, -1, 1, -1, 1],
        [-1, 1, -1, -1, 1, -1, 1],
        [-1, -1, 1, 1, -1, -1, 1],
        [1, 1, -1, 1, -1, -1, 1]
    ]).T

    stepsize = 0.1
    maxiter = 10000

    #X,y = gen_nonlinear_data(500, 40, 1)

    X, Xt, y, yt = train_test_split(X, y, test_size=0.2)
    X, y = pad(X, y, 7)
    Xt, yt = pad(Xt, yt, 7)

    Master_uncoded = master(X, None, 21)
    Master = master(X, G, 3)
    times_grid = np.zeros((5, 4))
    loss_grid = np.zeros((5, 4))  # Initialize for each dataset
    iters_grid = np.zeros((5, 4))
    test_loss = np.zeros((5, 4))
    test_loss_uncoded = np.zeros((5,4))
    uncoded_times = np.zeros((5, 4))
    # loss from normal logistic regression
    w0 = np.random.uniform(-1, 1, (X.shape[1], 1))
    start_time = time.time()
    w, num_iters = grdescentnormal(normallogistic, w0, stepsize, maxiter, Master_uncoded, y, X, tolerance=1e-02)
    normal_loss = get_loss(w, Xt, yt)
    end_time = time.time()
    print(f"loss from unquantized logistic regresison: {normal_loss} on {num_iters} iterations in {end_time - start_time} seconds")
    repetitions = 40
    for i in range(repetitions):
        for w_lvl in range(4, 9):
            for grd_lvl in range(2, 6):
                # logic for calculations
                w0 = np.random.uniform(-1, 1, (X.shape[1], 1))

                start_time = time.time()
                #w, num_iters = grdescentquant(func, w0, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, tolerance=1e-02)
                w, num_iters = grdescentquant(func, w0, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, 1e-02)
                # grdescentquant(func, w, stepsize, maxiter, Master, w_lvl, grd_lvl, X, y, filename, tolerance)

                end_time = time.time()

                times_grid[w_lvl - 4, grd_lvl - 2] += end_time - start_time
                loss_grid[w_lvl - 4, grd_lvl - 2] += get_loss(w, X, y)
                iters_grid[w_lvl - 4, grd_lvl - 2] += num_iters
                test_loss[w_lvl - 4, grd_lvl - 2] += get_loss(w, Xt, yt)

                start_time = time.time()
                w, num_iters = grdescentuncoded(uncoded_logistic, w0, stepsize, maxiter, Master_uncoded, w_lvl, grd_lvl, X, y, tolerance=1e-02)
                end_time = time.time()
                uncoded_times[w_lvl - 4, grd_lvl - 2] += end_time - start_time
                test_loss_uncoded[w_lvl - 4, grd_lvl - 2] += get_loss(w,Xt,yt)

    test_loss = test_loss/repetitions
    loss_grid = loss_grid/repetitions
    iters_grid = iters_grid/repetitions
    times_grid = times_grid/repetitions
    test_loss_uncoded = test_loss_uncoded/repetitions
    uncoded_times = uncoded_times/repetitions



    print(f" training loss: \n{loss_grid}")
    print(f" avg iterations: \n{iters_grid}")
    print(f" test loss: \n{test_loss}")
    print(f"run time: \n{times_grid}")
    print(f" test loss uncoded: \n{test_loss_uncoded}")
    print(f" times uncoded: \n{uncoded_times}")
    return test_loss, loss_grid, iters_grid, times_grid, test_loss_uncoded, uncoded_times

In [2]:
X, y = gen_nonlinear_data(500, 40, 1)

test_loss, loss_grid, iters_grid, times_grid, test_loss_uncoded, uncoded_times = run(X,y, "nonlinear_data.csv")

loss from unquantized logistic regresison: 0.09523809523809523 on 766 iterations in 0.133713960647583 seconds
 training loss: 
[[0.49115764 0.49440887 0.4941133  0.49261084]
 [0.48527094 0.49603448 0.48768473 0.49519704]
 [0.46667488 0.47665025 0.47864532 0.47007389]
 [0.44017241 0.43280788 0.44295567 0.44105911]
 [0.38115764 0.39012315 0.39906404 0.40428571]]
 avg iterations: 
[[260.97 230.09 239.61 232.37]
 [263.05 253.83 250.96 244.52]
 [287.34 274.78 263.89 269.15]
 [295.77 295.16 287.03 286.15]
 [329.33 301.16 298.96 302.41]]
 test loss: 
[[0.47342857 0.48552381 0.48266667 0.478     ]
 [0.48152381 0.49609524 0.4787619  0.49752381]
 [0.49590476 0.51266667 0.50371429 0.5012381 ]
 [0.50866667 0.48942857 0.50057143 0.49933333]
 [0.46638095 0.46180952 0.47247619 0.47580952]]
run time: 
[[2.90025017 3.36390747 4.30886775 4.96642804]
 [3.0502988  3.81617194 4.64595437 5.37977205]
 [3.45606411 4.27681323 5.01019179 6.06243483]
 [3.77089944 4.77500539 5.66077522 6.6024373 ]
 [4.39677491 5.

In [3]:
X, y = gen_data(500, 40, 1)
test_loss1, loss_grid1, iters_grid1, times_grid1, test_loss_uncoded1, uncoded_times1 = run(X,y, "linear_data.csv")

loss from unquantized logistic regresison: 0.08571428571428572 on 785 iterations in 0.17470216751098633 seconds
 training loss: 
[[0.50578818 0.5014532  0.5029803  0.5029064 ]
 [0.47985222 0.48605911 0.48913793 0.48418719]
 [0.36453202 0.38802956 0.38916256 0.37076355]
 [0.223867   0.24029557 0.24214286 0.23628079]
 [0.12066502 0.1096798  0.12342365 0.12123153]]
 avg iterations: 
[[183.07 182.49 196.79 191.37]
 [203.53 205.19 200.1  204.22]
 [229.71 226.37 238.19 236.42]
 [256.48 268.91 263.27 276.27]
 [290.5  298.79 305.87 309.98]]
 test loss: 
[[0.52409524 0.52742857 0.52761905 0.52742857]
 [0.5007619  0.516      0.50666667 0.50961905]
 [0.394      0.39971429 0.40152381 0.39771429]
 [0.28438095 0.28895238 0.29419048 0.28142857]
 [0.22895238 0.21704762 0.22352381 0.21828571]]
run time: 
[[2.14736312 2.80759262 3.71269566 4.31131374]
 [2.49037864 3.22775774 3.85938927 4.67238011]
 [2.90433239 3.68057764 4.72489969 5.55927186]
 [3.41165473 4.54880669 5.37001762 6.66250607]
 [4.13250809 

In [4]:

data = arff.loadarff('/Users/willem/Downloads/speeddating.arff')
df = pd.DataFrame(data[0])

hill_train_x, hill_test_x, hill_train_y, hill_test_y = clean_and_scale(df, "match")
hill_train_x, hill_train_y = pad(hill_train_x, hill_train_y, 7)
hill_test_x, hill_test_y = pad(hill_test_x, hill_test_y, 7)

print(hill_train_x.shape, hill_train_y.shape, hill_test_x.shape, hill_test_y.shape)

X = np.vstack((hill_test_x,hill_train_x))
y = np.vstack((hill_test_y, hill_train_y))
test_loss_real, loss_grid_real, iters_grid_real, times_grid_real, test_loss_uncoded_real, uncoded_times_real  = run(X,y, "real_data.csv")


(840, 238) (840, 1) (210, 238) (210, 1)
loss from unquantized logistic regresison: 0.3 on 281 iterations in 0.2311849594116211 seconds
 training loss: 
[[0.41913095 0.4642381  0.4437619  0.41917857]
 [0.40795238 0.41754762 0.41196429 0.41782143]
 [0.33577381 0.32122619 0.33252381 0.33469048]
 [0.22959524 0.22295238 0.21964286 0.22242857]
 [0.19058333 0.18736905 0.18085714 0.17980952]]
 avg iterations: 
[[211.09 211.58 217.97 215.47]
 [217.6  223.82 222.33 225.98]
 [225.1  223.52 227.81 224.77]
 [234.73 227.5  225.4  221.73]
 [247.71 236.7  237.17 240.99]]
 test loss: 
[[0.4427619  0.47419048 0.4597619  0.44280952]
 [0.42809524 0.43452381 0.42990476 0.43519048]
 [0.42519048 0.42033333 0.42447619 0.4217619 ]
 [0.40671429 0.39128571 0.38547619 0.38780952]
 [0.40004762 0.38242857 0.3747619  0.37671429]]
run time: 
[[13.66861801 19.5121558  25.95557152 31.51137056]
 [14.44581372 20.94453047 26.74991587 33.36401278]
 [15.28556747 21.26693627 27.80525244 33.61682624]
 [16.37931511 22.03998557

In [5]:
from gen_data import gen_seperable_data
X, y, w = gen_seperable_data(500, 40)

test_loss_sep, loss_grid_sep, iters_grid_sep, times_grid_sep, test_loss_uncoded_sep, uncoded_times_sep = run(X,y, "seperable_data.csv")

loss from unquantized logistic regresison: 0.05714285714285714 on 38 iterations in 0.008123159408569336 seconds


/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/uncoded_logistic.py:27: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:27: RuntimeWarning: overflow encountered in exp
  loss = np.mean(np.log(1 + np.exp(-vals)))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning

/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/uncoded_logistic.py:27: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/uncoded_logistic.py:27: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: o

/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/uncoded_logistic.py:27: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/uncoded_logistic.py:27: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/quant_logistic.py:30: RuntimeWarning: overflow encountered in exp
  func = lambda x: 1 / (1 + np.exp(x))
/Users/willem/Desktop/research/low_access-merge_tql/logistic_regression/uncoded_logistic.py:27: RuntimeWarning:

 training loss: 
[[0.04662562 0.07600985 0.02699507 0.01714286]
 [0.06618227 0.0564532  0.03674877 0.01719212]
 [0.05635468 0.0379803  0.04381773 0.03736453]
 [0.04036946 0.02512315 0.03115764 0.02519704]
 [0.03839901 0.01980296 0.01933498 0.02029557]]
 avg iterations: 
[[ 36.22  50.32  24.37  17.58]
 [ 66.26  41.06  33.69  24.5 ]
 [ 58.34 112.7   32.28  36.33]
 [232.92  58.62  59.69  53.96]
 [230.24 133.73 101.21  53.68]]
 test loss: 
[[0.07638095 0.10485714 0.05761905 0.04771429]
 [0.09542857 0.08580952 0.06666667 0.04819048]
 [0.08590476 0.06819048 0.07380952 0.06866667]
 [0.07104762 0.05533333 0.06057143 0.05638095]
 [0.06857143 0.05066667 0.05038095 0.05114286]]
run time: 
[[0.40313112 0.72597835 0.44601576 0.38206457]
 [0.76797913 0.61627314 0.6197948  0.53931224]
 [0.69539161 1.74683089 0.61787622 0.81327079]
 [2.93448324 0.93574624 1.16297018 1.24225024]
 [3.0533384  2.24399739 2.0421278  1.27208336]]
 test loss uncoded: 
[[0.07638095 0.10485714 0.05761905 0.04771429]
 [0.09542

In [ ]:
X, y = gen_nonlinear_data(2000, 200, 1)

test_losshd, loss_gridhd, iters_gridhd, times_gridhd, test_loss_uncodedhd, uncoded_timeshd = run(X,y, "hdnonlinear_data.csv")

loss from unquantized logistic regresison: 0.4630541871921182 on 10000 iterations in 9.997010231018066 seconds


In [ ]:
def plot_3d_bar(data):

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    rows, cols = data.shape
    x, y = np.meshgrid(np.arange(cols), np.arange(rows))

    x = x.flatten()
    y = y.flatten()
    z = np.zeros_like(x)

    dz = data.flatten()

    ax.bar3d(x, y, z, 1, 1, dz, shade=True)

    ax.set_xlabel('w_lvl')
    ax.set_ylabel('grd_lvl')
    ax.set_zlabel('z')

    plt.show()

In [ ]:
def plot_time_per_query(df):
    import numpy as np

    df_numeric = df.select_dtypes(include=[np.number])

    grouped = df_numeric.groupby(['w-quantization', 'grd-quantization']).mean()

    # Reset the index to make plotting easier
    grouped = grouped.reset_index()

    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    # Setting up the figure and 3D axis
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Data for the bars
    x = grouped['w-quantization']
    y = grouped['grd-quantization']
    z = grouped['time']  # or any other column you wish to visualize

    # The bar positions and width
    dx = dy = 0.5
    dz = z

    ax.bar3d(x, y, [0]*len(z), dx, dy, dz, color='b')

    # Labeling the axes
    ax.set_xlabel('w-quantization')
    ax.set_ylabel('grd-quantization')
    ax.set_zlabel('Average Time Per Query')

    # Show the plot
    plt.show()

In [ ]:
file_path = 'access_measurements.csv'

column_names = [
    'w-quantization', 'grd-quantization', 'imputation', 'access',
    'query type', 'time', 'stop cond', 'iters'
]

df_nonlinear = pd.read_csv(file_path, header=None, names=column_names)

print(df_nonlinear.head())

In [7]:
bad_w = [-0.08605444406989449, -0.06381547465695116, -0.04157650524400783, -0.019337535831064498, 0.0029014335818787276, 0.02514040299482216, 0.05849885711423698, 0.08073782652718031
         , 0.09185731123365215, 0.1140962806465955, 0.1363352500595388, 0.15857421947248213, 0.18081318888542547, 0.2030521582983688, 0.2252911277113121, 0.24753009712425544]

for i in range(1,len(bad_w)):
    print(f"\n {bad_w[i] - bad_w[i-1]}")






 0.02223896941294333

 0.02223896941294333

 0.02223896941294333

 0.022238969412943226

 0.022238969412943434

 0.03335845411941482

 0.02223896941294333

 0.011119484706471838

 0.022238969412943344

 0.022238969412943316

 0.022238969412943316

 0.022238969412943344

 0.022238969412943344

 0.022238969412943288

 0.022238969412943344
